In [38]:
using JLD,JLD2,Knet
using Base.Iterators: flatten

In [39]:
include("types.jl")
include("pre_processing.jl")
include("encoder_utils.jl")

fillvecs! (generic function with 1 method)

In [41]:
d = JLD.load("english_chmodel_updated.jld2")
v = create_vocab(d["new_d2"])
corpus = load_conllu("tr_imst-ud-train.conllu",v)
wmodel = makewmodel(d["new_d2"])
fillvecs!(wmodel, corpus, v)

KeyError: KeyError: key '\x00\x00\x00\x11' not found

In [53]:
d["new_d2"]

Dict{String,Any} with 13 entries:
  "back"       => Any[Float32[1.70814 0.787587 … -0.636378 -1.76298; 4.32346 5.…
  "word_vocab" => Dict("null"=>8668,"Secure"=>7932,"Puppy"=>9040,"progression"=…
  "char_vocab" => Dict('E'=>45,'«'=>94,'⁄'=>110,']'=>109,'Z'=>78,'B'=>37,'5'=>6…
  "eowchar"    => '\x00\x00\x00\x13'
  "sowchar"    => '\x00\x00\x00\x12'
  "forw"       => Any[Float32[1.47578 0.055072 … -0.2823 -1.58338; -1.95253 -0.…
  "eosword"    => "</s>"
  "sosword"    => "<s>"
  "soft"       => Array{Float32,2}[[-0.303084 -1.18525 … -0.281071 -0.2053; -0.…
  "unkword"    => "<unk>"
  "cembed"     => Float32[0.593419 0.190279 … 0.11003 -0.000173569; -0.236287 0…
  "char"       => Any[Float32[0.0708071 -0.456333 … 0.557484 0.094816; -1.61625…
  "unkchar"    => '\x00\x00\x00\x11'

In [4]:
ev = extend_vocab!(v, corpus)

UndefVarError: UndefVarError: v not defined

In [5]:
ev.xpostags

UndefVarError: UndefVarError: ev not defined

In [6]:
s = corpus[1]

UndefVarError: UndefVarError: corpus not defined

In [7]:
s.xpostag

UndefVarError: UndefVarError: s not defined

In [8]:
ev = extend_vocab!(v, corpus)

UndefVarError: UndefVarError: v not defined

In [9]:
createfeatvec!(corpus)

UndefVarError: UndefVarError: corpus not defined

In [10]:
padfeatvec!(corpus)

UndefVarError: UndefVarError: corpus not defined

In [11]:
function mlp(w,x)
    for i=1:2:length(w)-2
        x = relu.(w[i]*x .+ w[i+1])
    end
    return w[end-1]*x .+ w[end]
end

mlp (generic function with 1 method)

In [12]:
function create_input_matrix(corpus)
    input = []
    for i in 1:length(corpus)
        for j in 1:length(corpus[i].word)
            push!(input,corpus[i].cavec[j])
        end
    end
    return transpose(hcat(input...))
end

create_input_matrix (generic function with 1 method)

In [13]:
input = create_input_matrix(corpus)

UndefVarError: UndefVarError: corpus not defined

In [14]:
function findmaxnumofwords(corpus)
    max_ = 0
    for i in 1:length(corpus)
        if length(corpus[i].word) > max_
            max_ = length(corpus[i].word)
        end
    end
    return max_
end

findmaxnumofwords (generic function with 1 method)

In [15]:
totalnumofwords = size(input,1)
totalnumoffeatures = size(input,2)
#max number of words that can be head in a sentence, number of columns for head output
numofhead = findmaxnumofwords(corpus)

numofdeprel = length(keys(UDEPREL))

UndefVarError: UndefVarError: input not defined

In [16]:
totalnumofwords

UndefVarError: UndefVarError: totalnumofwords not defined

In [17]:
struct Linear; w; b; end
Linear(i::Int,o::Int,scale=0.01) = Linear(scale * randn(o,i), zeros(o))
batchsize = totalnumoffeatures
# This one allows instances to be defined using input and output sizes:
model_head = Linear(totalnumoffeatures,numofhead)

UndefVarError: UndefVarError: totalnumoffeatures not defined

In [18]:
function createoutputheadmatrix(corpus)
    outputhead = []
    for i in 1:length(corpus)
        push!(outputhead, corpus[i].head)
    end
    return vcat(outputhead...)
end

createoutputheadmatrix (generic function with 1 method)

In [19]:
outputhead = createoutputheadmatrix(corpus)

UndefVarError: UndefVarError: corpus not defined

In [20]:
(m::Linear)(x) = m.w * x  .+ m.b

In [21]:
ypred = model_head(transpose(input))

UndefVarError: UndefVarError: input not defined

In [22]:
accuracy(model_head,transpose(input),outputhead)

UndefVarError: UndefVarError: input not defined

In [23]:
function nll(scores, y)
    expscores = exp.(scores)
    probabilities = expscores ./ sum(expscores, dims=1)
    answerprobs = (probabilities[y[i],i] for i in 1:length(y))
    mean(-log.(answerprobs))
end

nll (generic function with 1 method)

In [24]:
(m::Linear)(x, y) = Knet.nll(m(x), y)
model_head(transpose(input),outputhead)

UndefVarError: UndefVarError: input not defined

In [25]:
v.cdict

UndefVarError: UndefVarError: v not defined

In [26]:
a ='i'

'i': ASCII/Unicode U+0069 (category Ll: Letter, lowercase)

In [27]:
a

'i': ASCII/Unicode U+0069 (category Ll: Letter, lowercase)

In [28]:
m = d.sowchar

ErrorException: type Dict has no field sowchar

In [29]:
A = Array{Float64}(undef, 2, 3)

2×3 Array{Float64,2}:
 9.88131e-324  6.94656e-310  4.94066e-324
 4.94066e-324  1.4822e-323   6.94656e-310

In [38]:
A[1] = [1.0 2.0 3.0]

MethodError: MethodError: Cannot `convert` an object of type Array{Float64,2} to an object of type Float64
Closest candidates are:
  convert(::Type{T<:Number}, !Matched::T<:Number) where T<:Number at number.jl:6
  convert(::Type{T<:Number}, !Matched::Number) where T<:Number at number.jl:7
  convert(::Type{T<:Number}, !Matched::Base.TwicePrecision) where T<:Number at twiceprecision.jl:250
  ...

In [29]:
function wordlstm(model, data, mask, embeddings)
    weight, bias = wforw(model), bforw(model)
    T = length(data)
    B = length(data[1])
    H = div(length(bias), 4)
    
    num = 

    if isa(weight, KnetArray)
        mask = map(KnetArray, mask)
    end

    wzero = fill!(similar(bias, H, B), 0)

    # forward lstm
    hidden = cell = wzero
    fhiddens = Array{Any}(T-2)  # fhiddens = Array(Any, T-2) : deprecated
    for t in 1:T-2
        (hidden, cell) = _lstm(weight, bias, hidden, cell, embeddings[:, data[t]]; mask=mask[t])
        fhiddens[t] = hidden
    end

UndefVarError: UndefVarError: m not defined

In [58]:
using Random,Distributions
dist = Normal()

┌ Info: Recompiling stale cache file /home/ilayda/.julia/compiled/v1.1/Distributions/xILW0.ji for Distributions [31c24e10-a181-5473-b8eb-7969acd0382f]
└ @ Base loading.jl:1184


Normal{Float64}(μ=0.0, σ=1.0)

In [59]:
a = rand(dist, (2,3))

2×3 Array{Float64,2}:
  0.556994  0.220333  0.965141
 -0.658875  0.17955   1.01849 

In [62]:
b = size(a[0])

BoundsError: BoundsError: attempt to access 2×3 Array{Float64,2} at index [0]

In [63]:
corpus

3685-element Array{Any,1}:
 Sanal parçacıklar sa bunların hiçbirini yapamazlar .                                                                                                                                                                                                                   
 Ona her şeyimi verdim .                                                                                                                                                                                                                                                
 Karşısında , pantolonu dizlerine dek ıslak , önlük torbası ham eriklerle dolu İbrahim dikiliyordu .                                                                                                                                                                    
 Nereden biliyorsunuz .                                                                                                                                                           